In [ ]:
conda install -c conda-forge pycudadecon cudatoolkit=10.2

In [ ]:
conda create --name cudadecon python
activate cudadecon
pip install jupyterlab
conda install -c conda-forge "napari[all]" 

pip insatall opencv-python
pip install zarr
conda install dask -c conda-forge


In [2]:
import napari
import ioMicro as io
import numpy as np
import glob,os,sys

In [3]:
def load_and_fit_file(fl,icol=-2,th=1500,plt_val=False):
    im = io.read_im(fl)
    im_ = np.array(im[icol],dtype=np.float32)
    im__ = io.norm_slice(im_,s=20)
    Xh=io.get_local_max(im__,th,delta=3,delta_fit=3,dbscan=True,return_centers=False)
    if plt_val:
        resc = 5
        std_ = np.std(im__[::resc,::resc,::resc])
        std_abs = std_*12
        print(std_abs)

        import napari
        viewer = napari.view_image(im__)
        viewer.add_points(Xh[:,:3],face_color=[0,0,0,0],size=20,edge_color='yellow',out_of_slice_display=True)
    return im_,Xh
def im_to_pix_stats(im_,Xh,start_ = (0,1000,1000),size_ = 300, sz=30):
    
    
    end_ = np.min([np.array(start_)+size_,im_.shape],axis=0)
    size_ = end_-start_
    XI = np.indices(size_).reshape([3,-1]).T
    XI_ = XI[::]+start_


    from scipy.spatial import cKDTree
    X = Xh[:,:3]
    tree = cKDTree(X)
    d,i = tree.query(XI_,distance_upper_bound=sz*2+1,workers =20)
    keep = d<np.inf
    XI__ = XI_[keep]
    X__ = X[i[keep]]
    Xdif = XI__-X__
    H_ = im_[tuple(XI__.T)]

    tree2 = cKDTree(Xdif) #this part is slow


    from tqdm import tqdm
    sz=30
    size = [2*sz+1]*3
    Xim = (np.indices(size).reshape([3,-1]).T-sz)
    Xim.shape

    res = tree2.query_ball_point(Xim,1,workers=20)
    H_ = im_[tuple(XI__.T)]
    im_PSF = np.array([np.mean(H_[r_]) for r_ in res]).reshape(size)
    im_NPSF = np.array([len(r_) for r_ in res]).reshape(size)
    #im_PSF = np.array([np.nanmean(H_[r_]) for r_ in tqdm(res)]).reshape(size)
    
    return im_PSF,im_NPSF


def get_psf(dic_psf,key = (0, 1200, 1200)):
    npix = len(dic_psf[key][0])
    num_pix = np.round(npix**(1/3)).astype(int)
    im_PSF = np.array([np.mean([h for hs in dic_psf[key] for h in hs[ipix]]) for ipix in np.arange(npix)]).reshape([num_pix]*3)
    return im_PSF

In [4]:
dic_psf = {}

In [10]:
fls = [r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__004.zarr',
       r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__006.zarr',
      r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__007.zarr',
      r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__009.zarr',
      r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__010.zarr',
      r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__012.zarr',
      r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__031.zarr',
      r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__033.zarr',
      r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__034.zarr',
      r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__037.zarr']
fls = [r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__038.zarr',
      r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\Conv_zscan__039.zarr']
#import napari
#napari.view_image(io.read_im(fls[-1]));

In [5]:
fls = glob.glob(r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set1\*.zarr')
#fls

In [6]:
#io.read_im(fls[-1])
dic_psf = {}

In [7]:
#import pickle
#dic_psf = pickle.load(open(r'C:\Users\Bogdan\Downloads\psf_Scope2_25x-1p5_cy3_shallow_small (1).pkl','rb'))

In [8]:
from tqdm import tqdm
for fl in fls[len(dic_psf.get((0,0,0),[])):]:
    im_,Xh = load_and_fit_file(fl,icol=-2,th=1500,plt_val=False)
    sz_sq=300
    indxyz = np.indices(np.ceil(np.array(im_.shape)/sz_sq).astype(int)).reshape([3,-1]).T*sz_sq
    for ind_ in tqdm(indxyz):
        ind__ = tuple(ind_)
        try:
            im_PSF,im_NPSF = im_to_pix_stats(im_,Xh,start_ = ind__,size_ = sz_sq, sz=30)
        except:
            print("Failed")
        if ind__ not in dic_psf: dic_psf[ind__]=[]
        dic_psf[ind__].append([im_PSF,im_NPSF])
    import pickle
    pickle.dump(dic_psf,open(r'Y:\YiqunPSF\psf_Scope2_25x-1p5_cy3_shallow_small.pkl','wb'))

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

...

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

### Scope 3 40X

In [26]:
dic_psf = {}

In [21]:
import glob
fls = glob.glob(r'\\132.239.197.16\Raw_data\DCBBL1_11_3_2022\H2_Igfbpl1,Aldh1l1,Ptbp1_set1\*.zarr')[22:]

In [25]:
fl = fls[10]
im_,Xh = load_and_fit_file(fl,icol=-2,th=2700,plt_val=True)

2352.1525268554688


In [27]:
from tqdm import tqdm
for fl in fls[len(dic_psf.get((0,0,0),[])):]:
    im_,Xh = load_and_fit_file(fl,icol=-2,th=2700,plt_val=False)
    sz_sq=300
    indxyz = np.indices(np.ceil(np.array(im_.shape)/sz_sq).astype(int)).reshape([3,-1]).T*sz_sq
    for ind_ in tqdm(indxyz):
        ind__ = tuple(ind_)
        try:
            im_PSF,im_NPSF = im_to_pix_stats(im_,Xh,start_ = ind__,size_ = sz_sq, sz=30)
        except:
            print("Failed")
        if ind__ not in dic_psf: dic_psf[ind__]=[]
        dic_psf[ind__].append([im_PSF,im_NPSF])
    import pickle
    pickle.dump(dic_psf,open('psf_Scope3_40x_cy3_shallow_small.pkl','wb'))

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

...

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
  0%|                                               

KeyboardInterrupt: 

In [ ]:
def get_psf(dic_psf,key = (0, 0, 0),sz_sm = 5,N=10,plt_val=True,thcor=0.92,slice_=slice(None)):

    XI = (np.indices([sz_sm,sz_sm,sz_sm])-(sz_sm//2)).reshape([3,-1]).T+np.array(dic_psf[key][0][0].shape)//2
    XI = tuple(XI.T)
    mat = np.zeros([len(dic_psf[key])]*2)
    for iim1,(im_PSF1,im_NPSF1) in enumerate(dic_psf[key]): 
        for iim2,(im_PSF2,im_NPSF2) in enumerate(dic_psf[key]):

            c = np.corrcoef(im_PSF1[XI],im_PSF2[XI])[0,1]
            if not np.isnan(c) and iim1!=iim2:
                mat[iim1,iim2]=c
    mat[mat==0]=np.nan
    cors = np.nanmedian(mat,axis=0)
    print(np.mean(cors>thcor))
    igoodfovs = np.where(cors>thcor)[0][slice_]

    im_Nmax = np.sum([dic_psf[key][igood][-1] for igood in igoodfovs],axis=0)
    im_PSFf = np.nansum([dic_psf[key][igood][0]*(dic_psf[key][igood][-1]/im_Nmax) for igood in igoodfovs],axis=0)
    im_PSFf[im_Nmax<N]=np.min(im_PSFf[im_Nmax>N])
    
    if plt_val:
        import napari
        if type(plt_val) is napari.viewer.Viewer:
            plt_val.add_image(im_PSFf)
        else:
            import napari
            napari.view_image(im_PSFf)
    return im_PSFf

In [ ]:
dic_psf = pickle.load(open('psf_Scope2_25x-1p5_cy3_shallow_small.pkl','rb'))

In [ ]:
data = get_psf(dic_psf,key = (0, 900, 900),sz_sm = 9,N=100,thcor=0.97,plt_val=False)

In [ ]:
fls = glob.glob(r'\\132.239.200.145\bbfishdc9\DCBBL1_12_06_2022\H1_Igfbpl1,Aldh1l1,Ptbp1_set2\*.zarr')
dic_psf = {}
from tqdm import tqdm
for fl in fls[len(dic_psf.get((0,0,0),[])):]:
    im_,Xh = load_and_fit_file(fl,icol=-2,th=1500,plt_val=False)
    sz_sq=300
    indxyz = np.indices(np.ceil(np.array(im_.shape)/sz_sq).astype(int)).reshape([3,-1]).T*sz_sq
    for ind_ in tqdm(indxyz):
        ind__ = tuple(ind_)
        im_PSF,im_NPSF = im_to_pix_stats(im_,Xh,start_ = ind__,size_ = sz_sq, sz=30)
        if ind__ not in dic_psf: dic_psf[ind__]=[]
        dic_psf[ind__].append([im_PSF,im_NPSF])
    import pickle
    pickle.dump(dic_psf,open('psf_Scope2_25x-1p5_cy3_shallow_small_set2.pkl','wb'))

In [32]:
key = (0, 1200, 1200)
im_Nmax = np.sum([im_NPSF for im_PSF,im_NPSF in dic_psf[key]],axis=0)
im_PSFf = np.nansum([im_PSF*(im_NPSF/im_Nmax) for im_PSF,im_NPSF in dic_psf[key]],axis=0)

import napari
napari.view_image(im_PSFf)

C:\Users\Bogdan\AppData\Local\Temp\ipykernel_33640\2987046038.py:3: RuntimeWarning: invalid value encountered in divide
  im_PSFf = np.nansum([im_PSF*(im_NPSF/im_Nmax) for im_PSF,im_NPSF in dic_psf[key]],axis=0)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 30.0, 30.0), zoom=6.69672131147541, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(30.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 61.0, 1.0), (0.0, 61.0, 1.0), (0.0, 61.0, 1.0)), current_step=(30, 30, 30), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im_PSFf' at 0x17da9962b50>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 

In [28]:
np.sum(im_Nmax)

6464088

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 30.0, 30.0), zoom=6.69672131147541, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(30.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 61.0, 1.0), (0.0, 61.0, 1.0), (0.0, 61.0, 1.0)), current_step=(30, 30, 30), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im_PSFf' at 0x17d98019350>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 

In [211]:

import napari
napari.view_image(get_psf(dic_psf,key = (0, 0, 1200)));

C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
#res,H_ = im_to_pix_stats(im_,Xh,start_ = ind__,size_ = sz_sq, sz=30,retrun_psf=False)

In [121]:
dic_psf = pickle.load(open('psf_Scope2_25x-1p5_cy3_shallow.pkl','rb'))

In [188]:
rT,HT = res[0],H_[0]

In [190]:
rT,HT

(array([list([]), list([]), list([]), ..., list([]), list([]), list([])],
       dtype=object),
 array([list([]), list([]), list([]), ..., list([]), list([]), list([])],
       dtype=object))

IndexError: index 246594 is out of bounds for axis 0 with size 226981

In [181]:
key = (0, 1200, 1200)
res__ = dic_psf[key]
[[H[ris[ipix]]for ris,H in res__] for ipix in range(len(res__[0][0]))]

[[array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([], dtype=float32), array([], dtype=float32)],
 [array([]

C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [150]:
dic_psf.keys()

dict_keys([(0, 0, 0), (0, 0, 300), (0, 0, 600), (0, 0, 900), (0, 0, 1200), (0, 0, 1500), (0, 0, 1800), (0, 0, 2100), (0, 0, 2400), (0, 0, 2700), (0, 300, 0), (0, 300, 300), (0, 300, 600), (0, 300, 900), (0, 300, 1200), (0, 300, 1500), (0, 300, 1800), (0, 300, 2100), (0, 300, 2400), (0, 300, 2700), (0, 600, 0), (0, 600, 300), (0, 600, 600), (0, 600, 900), (0, 600, 1200), (0, 600, 1500), (0, 600, 1800), (0, 600, 2100), (0, 600, 2400), (0, 600, 2700), (0, 900, 0), (0, 900, 300), (0, 900, 600), (0, 900, 900), (0, 900, 1200), (0, 900, 1500), (0, 900, 1800), (0, 900, 2100), (0, 900, 2400), (0, 900, 2700), (0, 1200, 0), (0, 1200, 300), (0, 1200, 600), (0, 1200, 900), (0, 1200, 1200), (0, 1200, 1500), (0, 1200, 1800), (0, 1200, 2100), (0, 1200, 2400), (0, 1200, 2700), (0, 1500, 0), (0, 1500, 300), (0, 1500, 600), (0, 1500, 900), (0, 1500, 1200), (0, 1500, 1500), (0, 1500, 1800), (0, 1500, 2100), (0, 1500, 2400), (0, 1500, 2700), (0, 1800, 0), (0, 1800, 300), (0, 1800, 600), (0, 1800, 900), (0,

In [143]:
import napari
napari.view_image(im_);

In [ ]:
def get_psf_from_stat():
    

In [ ]:
im_PSF = np.array([np.median(H_[r_]) for r_ in tqdm(res)]).reshape(size)

In [118]:
for ind_ in tqdm(indxyz):
    ind__ = tuple(ind_)
    res,H_ = im_to_pix_stats(im_,Xh,start_ = ind__,size_ = sz_sq, sz=30,retrun_psf=False)
    if ind__ not in dic_psf: dic_psf[ind__]=[]
    dic_psf[ind__].append((res,H_))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:38<00:00,  1.01it/s]


In [79]:
np.arange()

In [101]:
im_PSF = im_to_pix_stats(im_,Xh,start_ = (0,1000,1000),size_ = 300, sz=30,retrun_psf=True)

Exception ignored in: <function tqdm.__del__ at 0x000001FFF7057100>
Traceback (most recent call last):
  File "C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\tqdm\std.py", line 1162, in __del__
    self.close()
  File "C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\tqdm\notebook.py", line 288, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
100%|███████████████████████████████████████████████████████████████████████| 226981/226981 [00:06<00:00, 37582.31it/s]


In [102]:
napari.view_image(im_PSF)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 30.0, 30.0), zoom=6.69672131147541, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(30.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 61.0, 1.0), (0.0, 61.0, 1.0), (0.0, 61.0, 1.0)), current_step=(30, 30, 30), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im_PSF' at 0x2012762fd10>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: '

In [75]:
len(res)/61/61

61.0

In [72]:
import napari
napari.view_image(im_PSF)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 30.0, 30.0), zoom=6.69672131147541, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(30.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 61.0, 1.0), (0.0, 61.0, 1.0), (0.0, 61.0, 1.0)), current_step=(30, 30, 30), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im_PSF' at 0x200372d8f50>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: '